# 22장. 고급테크닉

## 이장에서 다루는내용
    - 고급함수
    - 쉽게 조작할 수 없는 객체
    - 타이머 조작

## 고급함수
- 자바스크립트에서 가장 흥미로운 부분인 함수이다. 자바스크립트 함수는 매우 단순하고 절차적인 방식으로 사용할 수도, 대단히 복잡하고 동적인 방식으로 사용할 수도 있다. 

- 클로저를 이용해 기능 추가도 할 수 있다.
- 함수는 결국 객체이므로 함수 포인터를 사용하기도 매우 쉽다.

### 안전한 타입탐지
- 자바스크립트 내장된 타입 탐지 메커니즘은 그다지 완벽하지 않으며, 혼란스러운 결과를 나타낼 때도 많다. 예를들어 typeof 연산자에는 여러 가지 문제가 있어서 데이터 타입을 신뢰할 수 없을때가 있다. 

- instanceof 연산자에도 문제가 있어서, 프레임을 사용한 문서처럼 전역 스코프가 여러개 있을때에는 신뢰할 수 없다. 이 문제의 고전적인 예제는 배열과 객체를 혼동하는 경우다.

In [ ]:
var isArray = value instanceof Array;

- 이 코드가 true를 반환하는 경우는 오직 value가 실제 배열일때
- 이 코드와 같은 전역 스코프에 존재하는 Array 생성자를 통해 생성되었을때
- value가 다른 프레임에서 생성된 배열이면 이 코드는 false 반환

### 네이티브
- 네이티브는 실행 환경에 종속되지 않는 ECMA Script 명세의 내장 객체를 뜻하며, 함수의 형태로 이루어져 있습니다.(ex, window 객체는 브라우저에 종속됩니다.)
- 다음은 자주 사용되는 네이티브입니다.
    - String()
    - Number()
    - Boolean()
    - Array()
    - Object()
    - function()
    - RegExp()
    - Date()
    - Error()
    - Symbol()
    
### Object 의 네이티브 toSting() 메서드

In [ ]:
var a = [1,2,3];
var b = /asdf/;

Object.prototype.toString.call(a); // '[object Array]'
Object.prototype.toString.call(b); // '[object RegExp]'

// 문득 드는 궁금증 
typeof a; // 'object'
a.toString(); // '1,2,3'

// 왜 이런 차이가?
Object.prototype.toString === a.toString; // false
Array.prototype.toString === a.toString; // true

// Array() toString이 Object() toString을 오버라이딩 한 것으로 추정

In [ ]:
// 배열을 어떤 전역 컨텍스트에서 생성했든 네이티브 생성자 이름은 항상 같으므로 toString()이 반환하는 값도 일정하다.
var a = [1,2,3];
function isArray(a) {
    return Object.prototype.toString.call(a) == "[object Array]";
}

In [ ]:
// 네이티브 함수인지 판단하는 함수
function isFunction(a) {
    return Object.prototype.toString.call(a) == "[object Function]";
}

In [ ]:
// 정규 표현식인지 판단 
function isRegExp(a) {
    return Object.prototype.toString.call(a) == "[object RegExp]";
}

## 스코프 확인 생성자

In [ ]:
// 생성자의 this 객체는 다음과 같이 새로 생성된 객체 인스턴스를 가리킴

function Person(name, age, job){
    this.name = name;
    this.age = age;
    this.job = job;
}

var person = new Person("Nicholas", 29, "Software Engineer");

- new 연산자와 함께 사용하면 새 Person 객체를 생성하고 프로퍼티를 할당하게 됨. new 연산자 없이 생성자 호출하면 문제발생

In [ ]:
// new 연산자 생략한채 생성자를 일반적인 함수처럼 호출한 결과 
var person = Person("Nicholas", 29, "FE developer");
console.log(window.name);
console.log(window.age);
console.log(window.job);

In [ ]:
function Person(name, age, job) {
    if (this instanceof Person){
        this.name = name;
        this.age = age;
        this.job = job;
    } else {
        return new Person(name, age, job);
    }
}

var person1 = Person("유리", 20, "유튜버");
console.log(window.name);
console.log(person1.job);

var person2 = Person("재현", 18, "페이스북직원");
console.log(window.name);
console.log(person2.job);

var person3 = new Person("세훈", 40, "부자");
console.log(person3.job);

In [ ]:
function Polygon(sides){
    if(this instanceof Polygon) {
        this.sides = sides;
        this.getArea = function(){
            return 0;
        };
    } else {
        return new Polygon(sides);
    }
}

function Rectangle(width, height) {
    Polygon.call(this, 2);
    this.width = width;
    this.height = height;
    this.getArea = function(){
        return this.width * this.height;
    }
}

Rectangle.prototype = new Polygon(); //생성자 훔치기 패턴을 쓸때 프로토 타입 체인이나 기생 상속을 함께 사용하면 됨

var rect = new Rectangle(5, 10);
console.log(rect.sides); // 2


## 함수바인딩

- 함수 바인딩이란 특정한 this 값과 특정한 매개변수를 넘기면서 다른 함수를 호출하는 함수
- 이 테크닉은 보통 콜백 및 이벤트 핸들러와 함께 사용해서 코드 실행 컨텍스트를 유지하면서도 함수를 변수처럼 전달하는 데 쓰인다

In [ ]:
var handler = { 
    message : "Event handled", 
    
    handleClick: function(event){ 
        console.log(this.message + ":" + event.type); 
    } 
}; 

var btn = document.getElementById("my-btn"); 
btn.addEventListener("click", handler.handleClick.bind(handler));


- 이 예제는 handler 라는 객체를 생성한다.
- handler.handleClick() 메서드는 DOM 버튼의 이벤트 핸들러로 등록된다. 

- undefined 호출이유는? 
    - 대부분의 브라우저에서 handler.handleClick()의 컨텍스트가 저장되지 않으므로 this 객체는 handler가 아니라 DOM 버튼을 가리킨다.

In [ ]:
// 위 예제를 클로저로 써서 수정한것
var handler = { 
    message : "Event handled", 
    
    handleClick: function(event){ 
        console.log(this.message); 
    } 
}; 

var btn = document.getElementById("my-btn"); 
EventUtil.addHandler(btn, "click", function(event){
    handler.handlerClick(event);
});

In [ ]:
function bind(fn, context){
    return function(){
        return fn.apply(context,arguments);
    };
}

// bind 함수 사용
var handler = { 
    message : "Event handled", 
    handleClick: function(event){ 
        console.log(this.message); 
    } 
}; 

var btn = document.getElementById("my-btn"); 
EventUtil.addHandler(btn, "click", bind(handlerClick, handler));
});

- 함수 바인딩은 함수 포인터를 반드시 값으로 전달해야하고 그 함수를 특정 컨텍스트에서 실행해야 할때 유용하다. 함수 바인딩은 대개 이벤트 핸들러에서 쓰이며 setTimeout()이나 setInterval()에도 사용한다.
- 바인드된 함수를 여러번 호출해야 하므로 메모리도 많이 사용하고 일반적인 함수에 비해 다소 느림

## 함수커링 
- 여러개의 인자(parameter)를 갖는 함수를 단일 인자를 갖는 함수들의 연결로 바꾸는 것


### 도대체 커링은 왜 해야하는것이며, 함수의 인자를 나눠서 얻는 이득이 무엇일까? 
- 가장 큰 장점은 역시나 재사용성이다. 함수를 만드는 이유는 여러가지가 있겠지만 가장 큰 이유는 다시 사용하기 위해서이다. 재사용할수록 생산성도 좋아지고 나중에 유지보수할때도 고쳐야할 부분이 적어지니 여러모로 좋은 코드를 만들 수 있다. 

### 커링전 함수와 커링함수 예시

In [ ]:
// 그냥 함수
let sum = function (x, y) {
    return x + y;
};

console.log(sum(5, 7));  // 12

In [ ]:
// 커링함수 x, y(여러개의 인자)를 단일 인자 x와 y를 가지는 함수로 나누면 된다.
let sum = function (x) {
    return function (y) {
        return x+y;
    }
};
console.log(sum(5)(7));  // 12

https://dev-momo.tistory.com/entry/Currying-in-Javascript

## 쉽게 조작할 수 없는 객체 (ECMA5에서 지원)

### 확장 불가능한 객체
- Object.preventExtensions() 으로 객체의 프로퍼티 확장을 막을 수 있다.
- Object.isExtensible()으로 확장 가능한지 탐지할 수 있다.


### 봉인된 객체
- Object.seal() 을 사용하면 확장할수도 지울수도 없다.
- isSealed()로 탐지


### 동결된 객체
- Object.freeze()로 확장이나, 봉인과 더불어 변경도 못하게 막는다.

## 고급 타이머

### 타이머 반복
- setTimeout, setInterval의 함수는 실행을 뒤로 미룬다. 중요한 것은 타이머는 코드가 실행될 시점이 아니라 큐에 추가될 시점을 말한다는 것.
- setTimeout은? 
    - 일정 시간 후에 특정 코드, 함수를 의도적으로 지연한 뒤 실행하고 싶을 때 사용하는 함수로 사용
    - 코드사용법: 
    setTimeout(function() { 
    // Code here 
    },delay);

### 프로세스 관리
- 오래 걸리는 스크립트의 경우 브라우저에서 제한함으로 타이머를 활용하면 회피할수 있다.


### 함수 감속
- 짧은시간동안 메모리 리소스를 많이 잡아먹는 작업을 너무 많이 하면 브라우저가 멈출 수 있다. 타이머를 통한 함수감속 패턴이 도움이 된다.




## 커스텀 이벤트

- 커스텀 이벤트 emitter를 사용하면 옵저버 패턴을 유용하게 사용할 수 있다.


## 드래그 앤 드롭
- mousedown, mousemove, mouseup 이벤트를 통하여 드래그앤 드롭을 구현할 수 있다
- 커스텀 이벤트를 만들어 사용하면 드래그앤 드롭의 상태를 파악하여 보여줄 수 있다.